### Import libraries

In [1]:
from IPython.core.display import display
from src.utils.preprocessing import classic_preprocessing
from src.utils.get_data import import_data, split_experts
from src.utils.train import hyperparameter_tuning_cv
from src.utils.config import *
import pandas as pd
import numpy as np

In [2]:
%load_ext autoreload
%autoreload 2

### Load Data

In [3]:
DATA_PATH = '../../../data'
X_fine, y_fine = import_data(DATA_PATH, segmentation_type='fine',
                             drop_user_features=False,
                             drop_expert=True)

In [4]:
# For each expert separately
X_e, y_e = import_data(DATA_PATH, segmentation_type='fine',
                       drop_user_features=False,
                       drop_expert=False)

In [5]:
display(X_fine.head())
display(y_fine.head())

EEPD50_100  EEPD100_150  \
subject                              file_id                            
008ba489-31ad-44d8-856b-fcf72369dc46 0               6.0          6.0   
                                     1               3.0          3.0   
                                     2               4.0          4.0   
                                     3               4.0          4.0   
                                     4               4.0          4.0   

                                              EEPD150_200  EEPD200_250  \
subject                              file_id                             
008ba489-31ad-44d8-856b-fcf72369dc46 0                5.0          4.0   
                                     1                3.0          4.0   
                                     2                3.0          4.0   
                                     3                4.0          4.0   
                                     4                5.0          4.0   

                                              EEPD250_300  EEPD300_350  \
subject                              file_id                             
008ba489-31ad-44d8-856b-fcf72369dc46 0                5.0          5.0   
                                     1                4.0          5.0   
                                     2                3.0          3.0   
                                     3                4.0          5.0   
                                     4                4.0          5.0   

                                              EEPD350_400  EEPD400_450  \
subject                              file_id                             
008ba489-31ad-44d8-856b-fcf72369dc46 0                6.0          6.0   
                                     1                4.0          4.0   
                                     2                4.0          4.0   
                                     3                3.0          4.0   
                                     4                3.0          4.0   

                                              EEPD450_500  EEPD500_550  ...  \
subject                              file_id                            ...   
008ba489-31ad-44d8-856b-fcf72369dc46 0                5.0          6.0  ...   
                                     1                4.0          3.0  ...   
                                     2                4.0          4.0  ...   
                                     3                4.0          3.0  ...   
                                     4                4.0          3.0  ...   

                                              PSD_225-425  PSD_450-550  \
subject                              file_id                             
008ba489-31ad-44d8-856b-fcf72369dc46 0           0.398299     0.026407   
                                     1           0.230052     0.086242   
                                     2           0.105747     0.180344   
                                     3           0.235517     0.121967   
                                     4           0.230394     0.119689   

                                              PSD_1325-1600  PSD_1600-1900  \
subject                              file_id                                 
008ba489-31ad-44d8-856b-fcf72369dc46 0             0.013356       0.033366   
                                     1             0.023806       0.008824   
                                     2             0.034701       0.006258   
                                     3             0.061882       0.002742   
                                     4             0.069418       0.002717   

                                              PSD_2500-2900  PSD_3100-3700  \
subject                              file_id                                 
008ba489-31ad-44d8-856b-fcf72369dc46 0             0.043195       0.010385   
                                     1             0.056102       0.011429   
                                     2       

Label
subject                              file_id       
008ba489-31ad-44d8-856b-fcf72369dc46 0          1.0
                                     1          1.0
                                     2          1.0
                                     3          1.0
                                     4          1.0

### Preprocessing

In [6]:
X_fine = classic_preprocessing(X_fine, norm=False)

In [7]:
# Save the expert feature for split
expert = X_e['Expert'].copy()
X_e = classic_preprocessing(X_e, norm=False)
X_e['Expert'] = expert.values

### Split expert models

In [8]:
X_e_1, y_e_1, X_e_2, y_e_2, X_e_3, y_e_3 = split_experts(X_e, y_e)

### Grid search

#### 1. Logistic regression

In [9]:
log_results = hyperparameter_tuning_cv(model='logistic', data=X_fine, labels=y_fine.Label, cv_k=5,
                                       params=LOGISTIC_PARAMS)

display(log_results)

f1_score  roc_auc_score  accuracy_score
max_iter oversampling                                         
10000    True          0.412988       0.567085        0.569757
         False         0.018266       0.502426        0.729624
100000   True          0.412988       0.567085        0.569757
         False         0.018266       0.502426        0.729624

In [10]:
# Best model parameters

best_log = log_results.iloc[[log_results.reset_index()['f1_score'].idxmax()]]

display(best_log)

,,f1_score,roc_auc_score,accuracy_score
max_iter,oversampling,,,
10000,True,0.412988,0.567085,0.569757


In [11]:
# for each expert
log_results_1 = hyperparameter_tuning_cv(model='logistic', data=X_e_1, labels=y_e_1, cv_k=5,
                                         params=LOGISTIC_PARAMS)
log_results_2 = hyperparameter_tuning_cv(model='logistic', data=X_e_2, labels=y_e_2, cv_k=5,
                                         params=LOGISTIC_PARAMS)
log_results_3 = hyperparameter_tuning_cv(model='logistic', data=X_e_3, labels=y_e_3, cv_k=5,
                                         params=LOGISTIC_PARAMS)

In [12]:
# Best model parameters

best_log_1 = log_results_1.iloc[[
    log_results_1.reset_index()['roc_auc_score'].idxmax()]]
best_log_2 = log_results_2.iloc[[
    log_results_2.reset_index()['roc_auc_score'].idxmax()]]
best_log_3 = log_results_3.iloc[[
    log_results_3.reset_index()['roc_auc_score'].idxmax()]]

display(best_log_1, best_log_2, best_log_3)

,,f1_score,roc_auc_score,accuracy_score
max_iter,oversampling,,,
10000,True,0.378743,0.677047,0.674554


,,f1_score,roc_auc_score,accuracy_score
max_iter,oversampling,,,
10000,True,0.323139,0.565205,0.585564


,,f1_score,roc_auc_score,accuracy_score
max_iter,oversampling,,,
10000,False,0.607526,0.646554,0.650483


#### 2. Linear Discriminant Analysis

In [13]:
lda_results = hyperparameter_tuning_cv(model='lda', data=X_fine, labels=y_fine.Label, cv_k=5,
                                       params=LDA_PARAMS)

display(lda_results)

,f1_score,roc_auc_score,accuracy_score
oversampling,,,
True,0.374699,0.568242,0.651260
False,0.166116,0.532169,0.731526


In [14]:
# Best model parameters

best_lda = lda_results.iloc[[lda_results.reset_index()['f1_score'].idxmax()]]

display(best_lda)

,f1_score,roc_auc_score,accuracy_score
oversampling,,,
True,0.374699,0.568242,0.65126


In [15]:
# For each expert
lda_results_1 = hyperparameter_tuning_cv(model='lda', data=X_e_1, labels=y_e_1, cv_k=5,
                                         params=LDA_PARAMS)
lda_results_2 = hyperparameter_tuning_cv(model='lda', data=X_e_2, labels=y_e_2, cv_k=5,
                                         params=LDA_PARAMS)
lda_results_3 = hyperparameter_tuning_cv(model='lda', data=X_e_3, labels=y_e_3, cv_k=5,
                                         params=LDA_PARAMS)

In [16]:
best_lda_1 = lda_results_1.iloc[[
    lda_results_1.reset_index()['roc_auc_score'].idxmax()]]
best_lda_2 = lda_results_2.iloc[[
    lda_results_2.reset_index()['roc_auc_score'].idxmax()]]
best_lda_3 = lda_results_3.iloc[[
    lda_results_3.reset_index()['roc_auc_score'].idxmax()]]

display(best_lda_1, best_lda_2, best_lda_3)

,f1_score,roc_auc_score,accuracy_score
oversampling,,,
True,0.349083,0.631606,0.732253


,f1_score,roc_auc_score,accuracy_score
oversampling,,,
True,0.241579,0.521757,0.672703


,f1_score,roc_auc_score,accuracy_score
oversampling,,,
True,0.642293,0.662332,0.663448


#### 3. K-nearest Neighbors

In [17]:
knn_results = hyperparameter_tuning_cv(model='knn', data=X_fine, labels=y_fine.Label, cv_k=5,
                                       params=KNN_PARAMS)

display(knn_results)

f1_score  roc_auc_score  accuracy_score
n_neighbors oversampling                                         
1           True          0.336058       0.519485        0.574608
            False         0.297966       0.518941        0.619971
2           True          0.275333       0.511873        0.626153
            False         0.125618       0.502653        0.697004
3           True          0.347361       0.513996        0.542463
            False         0.231637       0.507660        0.651736
4           True          0.300677       0.505864        0.587066
            False         0.128080       0.506394        0.702330
5           True          0.358493       0.516266        0.528864
            False         0.192536       0.506308        0.672943
6           True          0.328879       0.516417        0.577080
            False         0.107415       0.507036        0.710984
7           True          0.358652       0.514388        0.523348
            False         0.172732       0.512951        0.694817
8           True          0.333482       0.512579        0.560628
            False         0.092781       0.505914        0.714123
9           True          0.358523       0.512413        0.518117
            False         0.138495       0.510564        0.705278
10          True          0.339105       0.514137        0.556443
            False         0.075582       0.505172        0.718307
11          True          0.356457       0.509735        0.514788
            False         0.102923       0.505375        0.709748
12          True          0.335406       0.508775        0.548169
            False         0.059042       0.504549        0.722206
13          True          0.357656       0.508705        0.509558
            False         0.086177       0.505470        0.715454
14          True          0.343405       0.511971        0.543985
            False         0.043727       0.501245        0.721350
15          True          0.360199       0.510708        0.510319
            False         0.070985       0.504655        0.718878

In [18]:
# Best model parameters

best_knn = knn_results.iloc[[knn_results.reset_index()['f1_score'].idxmax()]]

display(best_knn)

,,f1_score,roc_auc_score,accuracy_score
n_neighbors,oversampling,,,
15,True,0.360199,0.510708,0.510319


In [19]:
knn_results_1 = hyperparameter_tuning_cv(
    model='knn', data=X_e_1, labels=y_e_1, cv_k=5, params=KNN_PARAMS)
knn_results_2 = hyperparameter_tuning_cv(
    model='knn', data=X_e_2, labels=y_e_2, cv_k=5, params=KNN_PARAMS)
knn_results_3 = hyperparameter_tuning_cv(
    model='knn', data=X_e_3, labels=y_e_3, cv_k=5, params=KNN_PARAMS)

In [ ]:
best_knn_1 = knn_results_1.iloc[[
    knn_results_1.reset_index()['roc_auc_score'].idxmax()]]
best_knn_2 = knn_results_2.iloc[[
    knn_results_2.reset_index()['roc_auc_score'].idxmax()]]
best_knn_3 = knn_results_3.iloc[[
    knn_results_3.reset_index()['roc_auc_score'].idxmax()]]

display(best_knn_1, best_knn_2, best_knn_3)

#### 4. Support Vector Classifier

In [ ]:
svc_results = hyperparameter_tuning_cv(model='svc', data=X_fine, labels=y_fine.Label, cv_k=5,
                                       params=SVC_PARAMS)

display(svc_results)

In [ ]:
# Best model parameters

best_svc = svc_results.iloc[[svc_results.reset_index()['f1_score'].idxmax()]]

display(best_svc)

In [ ]:
svc_results_1 = hyperparameter_tuning_cv(
    model='svc', data=X_e_1, labels=y_e_1, cv_k=5, params=SVC_PARAMS)
svc_results_2 = hyperparameter_tuning_cv(
    model='svc', data=X_e_2, labels=y_e_2, cv_k=5, params=SVC_PARAMS)
svc_results_3 = hyperparameter_tuning_cv(
    model='svc', data=X_e_3, labels=y_e_3, cv_k=5, params=SVC_PARAMS)

In [ ]:
best_svc_1 = svc_results_1.iloc[[
    svc_results_1.reset_index()['roc_auc_score'].idxmax()]]
best_svc_2 = svc_results_2.iloc[[
    svc_results_2.reset_index()['roc_auc_score'].idxmax()]]
best_svc_3 = svc_results_3.iloc[[
    svc_results_3.reset_index()['roc_auc_score'].idxmax()]]


display(best_svc_1, best_svc_2, best_svc_3)

#### 5. Naive Bayes Classifier

In [ ]:
nb_results = hyperparameter_tuning_cv(model='naive_bayes', data=X_fine, labels=y_fine.Label, cv_k=5,
                                      params=NAIVE_BAYES_PARAMS)

display(nb_results)

In [ ]:
# Best model parameters

best_nb = nb_results.iloc[[nb_results.reset_index()['f1_score'].idxmax()]]

display(best_nb)

In [ ]:
nb_results_1 = hyperparameter_tuning_cv(
    model='naive_bayes', data=X_e_1, labels=y_e_1, cv_k=5, params=NAIVE_BAYES_PARAMS)
nb_results_2 = hyperparameter_tuning_cv(
    model='naive_bayes', data=X_e_2, labels=y_e_2, cv_k=5, params=NAIVE_BAYES_PARAMS)
nb_results_3 = hyperparameter_tuning_cv(
    model='naive_bayes', data=X_e_3, labels=y_e_3, cv_k=5, params=NAIVE_BAYES_PARAMS)

In [ ]:
best_nb_1 = nb_results_1.iloc[[
    nb_results_1.reset_index()['roc_auc_score'].idxmax()]]
best_nb_2 = nb_results_2.iloc[[
    nb_results_2.reset_index()['roc_auc_score'].idxmax()]]
best_nb_3 = nb_results_3.iloc[[
    nb_results_3.reset_index()['roc_auc_score'].idxmax()]]

display(best_nb_1, best_nb_2, best_nb_3)

#### 6. Decision Tree

In [ ]:
dt_results = hyperparameter_tuning_cv(model='decision_tree', data=X_fine, labels=y_fine.Label, cv_k=5,
                                      params=DECISION_TREE_PARAMS)

display(dt_results)

In [ ]:
# Best model parameters

best_dt = dt_results.iloc[[dt_results.reset_index()['f1_score'].idxmax()]]

display(best_dt)

In [ ]:
dt_results_1 = hyperparameter_tuning_cv(
    model='decision_tree', data=X_e_1, labels=y_e_1, cv_k=5, params=DECISION_TREE_PARAMS)
dt_results_2 = hyperparameter_tuning_cv(
    model='decision_tree', data=X_e_2, labels=y_e_2, cv_k=5, params=DECISION_TREE_PARAMS)
dt_results_3 = hyperparameter_tuning_cv(
    model='decision_tree', data=X_e_3, labels=y_e_3, cv_k=5, params=DECISION_TREE_PARAMS)

In [ ]:
best_dt_1 = dt_results_1.iloc[[
    dt_results_1.reset_index()['roc_auc_score'].idxmax()]]
best_dt_2 = dt_results_2.iloc[[
    dt_results_2.reset_index()['roc_auc_score'].idxmax()]]
best_dt_3 = dt_results_3.iloc[[
    dt_results_3.reset_index()['roc_auc_score'].idxmax()]]

display(best_dt_1, best_dt_2, best_dt_3)

#### 7. Random Forest

In [ ]:
rf_results = hyperparameter_tuning_cv(model='random_forest', data=X_fine, labels=y_fine.Label, cv_k=5,
                                      params=RANDOM_FOREST_PARAMS)

display(rf_results)

In [ ]:
# Best model parameters

best_rf = rf_results.iloc[[rf_results.reset_index()['f1_score'].idxmax()]]

display(best_rf)

In [ ]:
rf_results_1 = hyperparameter_tuning_cv(
    model='random_forest', data=X_e_1, labels=y_e_1, cv_k=5, params=RANDOM_FOREST_PARAMS)
rf_results_2 = hyperparameter_tuning_cv(
    model='random_forest', data=X_e_2, labels=y_e_2, cv_k=5, params=RANDOM_FOREST_PARAMS)
rf_results_3 = hyperparameter_tuning_cv(
    model='random_forest', data=X_e_3, labels=y_e_3, cv_k=5, params=RANDOM_FOREST_PARAMS)

In [ ]:
best_rf_1 = rf_results_1.iloc[[
    rf_results_1.reset_index()['roc_auc_score'].idxmax()]]
best_rf_2 = rf_results_2.iloc[[
    rf_results_2.reset_index()['roc_auc_score'].idxmax()]]
best_rf_3 = rf_results_3.iloc[[
    rf_results_3.reset_index()['roc_auc_score'].idxmax()]]

display(best_rf_1, best_rf_2, best_rf_3)

#### 8. Gradient Boosting

In [ ]:
gb_results = hyperparameter_tuning_cv(model='gradient_boosting', data=X_fine, labels=y_fine.Label, cv_k=5,
                                      params=GRADIENT_BOOSTING_PARAMS)

display(gb_results)

In [ ]:
# Best model parameters

best_gb = gb_results.iloc[[gb_results.reset_index()['f1_score'].idxmax()]]

display(best_gb)

In [ ]:
gb_results_1 = hyperparameter_tuning_cv(
    model='gradient_boosting', data=X_e_1, labels=y_e_1, cv_k=5, params=GRADIENT_BOOSTING_PARAMS)
gb_results_2 = hyperparameter_tuning_cv(
    model='gradient_boosting', data=X_e_2, labels=y_e_2, cv_k=5, params=GRADIENT_BOOSTING_PARAMS)
gb_results_3 = hyperparameter_tuning_cv(
    model='gradient_boosting', data=X_e_3, labels=y_e_3, cv_k=5, params=GRADIENT_BOOSTING_PARAMS)

In [ ]:
best_gb_1 = gb_results_1.iloc[[
    gb_results_1.reset_index()['roc_auc_score'].idxmax()]]
best_gb_2 = gb_results_2.iloc[[
    gb_results_2.reset_index()['roc_auc_score'].idxmax()]]
best_gb_3 = gb_results_3.iloc[[
    gb_results_3.reset_index()['roc_auc_score'].idxmax()]]
display(best_gb_1, best_gb_2, best_gb_3)

### Results

In [ ]:
#display('logistic', best_log)
#display('lda', best_lda)
#display('knn', best_knn)
#display('svc', best_svc)
#display('naive_bayes', best_nb)
#display('decision_tree', best_dt)
#display('random_forest', best_rf)
#display('gradient_boosting', best_gb)

In [ ]:
weighted_average = (np.array([len(X_e_1), len(X_e_2), len(X_e_3)]) / len(X_e))

e_log = np.sum(pd.concat([best_log_1, best_log_2, best_log_3]
                         ).roc_auc_score.values * weighted_average)
e_lda = np.sum(pd.concat([best_lda_1, best_lda_2, best_lda_3]
                         ).roc_auc_score.values * weighted_average)
e_knn = np.sum(pd.concat([best_knn_1, best_knn_2, best_knn_3]
                         ).roc_auc_score.values * weighted_average)
e_svc = np.sum(pd.concat([best_svc_1, best_svc_2, best_svc_3]
                         ).roc_auc_score.values * weighted_average)
e_nb = np.sum(pd.concat([best_nb_1, best_nb_2, best_nb_3]
                        ).roc_auc_score.values * weighted_average)
e_dt = np.sum(pd.concat([best_dt_1, best_dt_2, best_dt_3]
                        ).roc_auc_score.values * weighted_average)
e_rf = np.sum(pd.concat([best_rf_1, best_rf_2, best_rf_3]
                        ).roc_auc_score.values * weighted_average)
e_gb = np.sum(pd.concat([best_gb_1, best_gb_2, best_gb_3]
                        ).roc_auc_score.values * weighted_average)

In [ ]:
# Results expert features
# First row is Expert 1, second Expert 2, third Expert 3
#display('logistic', pd.concat([best_log_1,best_log_2,best_log_3]))
#display('lda', pd.concat([best_lda_1,best_lda_2,best_lda_3]))
display('knn', pd.concat([best_knn_1, best_knn_2, best_knn_3]))
#display('svc', pd.concat([best_svc_1,best_svc_2,best_svc_3]))
#display('naive_bayes', pd.concat([best_nb_1,best_nb_2,best_nb_3]))
#display('decision_tree', pd.concat([best_dt_1,best_dt_2,best_dt_3]))
#display('random_forest', pd.concat([best_rf_1,best_rf_2,best_rf_3]))
#display('gradient_boosting', pd.concat([best_gb_1,best_gb_2,best_gb_3]))

In [ ]:
results = pd.DataFrame(data={'models': ['logistic', 'lda', 'knn', 'svc', 'naive_bayes', 'decision_tree',
                                        'random_forest', 'gradient_boosting'],
                             'auc_best': [best_log.roc_auc_score.values[0], best_lda.roc_auc_score.values[0],
                                          best_knn.roc_auc_score.values[0], best_svc.roc_auc_score.values[0],
                                          best_nb.roc_auc_score.values[0], best_dt.roc_auc_score.values[0],
                                          best_rf.roc_auc_score.values[0], best_gb.roc_auc_score.values[0]],
                             'auc_expert_weighted': [e_log, e_lda, e_knn, e_svc, e_nb, e_dt, e_rf, e_gb]})

display(results)

### Conclusions


In [ ]:
# Save dataframe
# results.to_pickle("results_fine_metadata.pkl")

In [ ]:
results.mean(axis=0)

In [ ]:
results.loc[results['auc_best'].argmax()]

In [ ]:
results.loc[results['auc_expert_weighted'].argmax()]

## ---------------------------------------

In [ ]:
# results.mean(axis=0)

In [ ]:
results.loc[results['auc_best'].argmax()]

In [ ]:
results.loc[results['auc_expert_weighted'].argmax()]